# Open Access status for WoS DOIs

Notebook used for the study presented at the EAHIL 2022 conference

* Title: Computational assistance in the analysis of cited references in biomedical literature: a case study from two institutions.

* Authors:
 * Teresa Lee, Knowledge Manager, International Agency for Research on Cancer (IARC/WHO) leet@iarc.fr  
 * Pablo Iriarte, IT Coordinator, Library of the University of Geneva Pablo.Iriarte@unige.ch 
 * Floriane Muller, Librarian (Medical Library), Library of the University of Geneva Floriane.Muller@unige.ch  
 * Ramon Cierco Jimenez, Doctoral Student, International Agency for Research on Cancer (IARC/WHO) CiercoR@students.iarc.fr  


### Prerequisites
* DOIs from publications and citations

## API unpaywall

https://unpaywall.org/products/api

### Schema

https://unpaywall.org/data-format

### Authentication

Requests must include your email as a parameter at the end of the URL, like this: api.unpaywall.org/my/request?email=YOUR_EMAIL.

### Rate limits

Please limit use to 100,000 calls per day

### Exemple

https://api.unpaywall.org/v2/10.1038/nature12373?email=YOUR_EMAIL


In [1]:
import re
import os
import pandas as pd
import time
import datetime
import requests

## Extract DOIs for UNIGE


In [2]:
# open UNIGE data
unige_citations = pd.read_csv('WoS UNIGE results 2001_2020/UNIGE_WoS.csv', encoding='utf-8', header=0, usecols=['Accession Number', 'DOI', 'DOI of cited article'])
unige_citations

,Accession Number,DOI,DOI of cited article
0,WOS:000307585400002,10.1111/j.1741-2358.2011.00508.x,NaN
1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,NaN
2,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,10.1111/j.1600-0528.1999.tb02031.x
3,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,10.1177/00220345990780020301
4,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,10.1111/j.1741-2358.2001.00079.x
...,...,...,...
1260976,WOS:000591530900014,10.1093/chemse/bjaa041,NaN
1260977,WOS:000591530900014,10.1093/chemse/bjaa041,10.1002/alr.22577
1260978,WOS:000591530900014,10.1093/chemse/bjaa041,10.1002/alr.22592
1260979,WOS:000591530900014,10.1093/chemse/bjaa041,10.1109/TPAMI.2020.2975798


In [3]:
# test values
unige_citations.loc[unige_citations['DOI of cited article'].isna()]

,Accession Number,DOI,DOI of cited article
0,WOS:000307585400002,10.1111/j.1741-2358.2011.00508.x,NaN
1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,NaN
7,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,NaN
8,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,NaN
14,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,NaN
...,...,...,...
1260969,WOS:000591530900014,10.1093/chemse/bjaa041,NaN
1260971,WOS:000591530900014,10.1093/chemse/bjaa041,NaN
1260972,WOS:000591530900014,10.1093/chemse/bjaa041,NaN
1260973,WOS:000591530900014,10.1093/chemse/bjaa041,NaN


In [5]:
# unige publications
unige_publications = unige_citations[['Accession Number', 'DOI']].drop_duplicates(subset='Accession Number')
unige_publications

,Accession Number,DOI
0,WOS:000307585400002,10.1111/j.1741-2358.2011.00508.x
1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x
48,WOS:000579849900001,10.1111/ger.12502
83,WOS:000308547000129,10.1111/j.1741-2358.2011.00586.x
119,WOS:000331109200007,10.1111/ger.12083
...,...,...
1260758,WOS:000459791000009,10.1111/bdi.12659
1260790,WOS:000538022300007,10.21037/atm-20-3754
1260869,WOS:000547552700033,10.21037/atm-2020-24
1260870,WOS:000580650400007,10.1016/j.jvs.2020.06.112


In [6]:
# test values
unige_publications.loc[unige_publications['DOI'].isna()]

,Accession Number,DOI
1333,WOS:000169353000022,NaN
1391,WOS:000232474700055,NaN
1392,WOS:000232474700033,NaN
2158,WOS:000238890200035,NaN
2209,WOS:000238890200033,NaN
...,...,...
1257634,WOS:000415152500005,NaN
1259442,WOS:000448188000495,NaN
1259552,WOS:000606080100488,NaN
1260048,WOS:000379749000250,NaN


In [8]:
# open IARC data
iarc_citations = pd.read_csv('WoS IARC results 2001_2020/IARC_WoS.csv', encoding='utf-8', header=0, usecols=['Accession Number', 'DOI', 'DOI of cited article'])
iarc_citations

,Accession Number,DOI,DOI of cited article
0,WOS:000492162206048,NaN,NaN
1,WOS:000438342200039,10.1093/ije/dyy122,10.1093/ije/dyy063
2,WOS:000443381100007,10.19191/EP18.1.P009.008,10.1093/ije/dyg257
3,WOS:000443381100007,10.19191/EP18.1.P009.008,NaN
4,WOS:000443381100007,10.19191/EP18.1.P009.008,NaN
...,...,...,...
275617,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN
275618,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN
275619,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN
275620,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN


In [9]:
# test values
iarc_citations.loc[iarc_citations['DOI of cited article'].isna()]

,Accession Number,DOI,DOI of cited article
0,WOS:000492162206048,NaN,NaN
3,WOS:000443381100007,10.19191/EP18.1.P009.008,NaN
4,WOS:000443381100007,10.19191/EP18.1.P009.008,NaN
6,WOS:000417001900034,10.1016/S1470-2045(17)30832-X,NaN
9,WOS:000417001900034,10.1016/S1470-2045(17)30832-X,NaN
...,...,...,...
275615,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN
275617,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN
275618,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN
275619,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN


In [10]:
# iarc publications
iarc_publications = iarc_citations[['Accession Number', 'DOI']].drop_duplicates(subset='Accession Number')
iarc_publications

,Accession Number,DOI
0,WOS:000492162206048,NaN
1,WOS:000438342200039,10.1093/ije/dyy122
2,WOS:000443381100007,10.19191/EP18.1.P009.008
5,WOS:000417001900034,10.1016/S1470-2045(17)30832-X
20,WOS:000399225400009,10.1007/s40944-017-0100-x
...,...,...
275178,WOS:000365993200031,10.1016/S0140-6736(15)00128-2
275264,WOS:000418101000024,10.1016/S0140-6736(17)32129-3
275318,WOS:000467473600049,10.1038/s41586-019-1171-x
275376,WOS:000385285000007,10.1016/S0140-6736(16)31012-1


In [11]:
# test values
iarc_publications.loc[iarc_publications['DOI'].isna()]

,Accession Number,DOI
0,WOS:000492162206048,NaN
73,WOS:000374988800051,NaN
165,WOS:000298011900887,NaN
665,WOS:000546012200002,NaN
942,WOS:000208590200011,NaN
...,...,...
238468,WOS:000224453700009,NaN
244124,WOS:000457544100001,NaN
250275,WOS:000324548204017,NaN
253201,WOS:000522625000007,NaN


In [12]:
# % of UNIGE publications with DOIs
1 - (unige_publications.loc[unige_publications['DOI'].isna()].shape[0] / unige_publications.shape[0])

0.8076986527357712

In [13]:
# % of UNIGE citations with DOIs
1 - (unige_citations.loc[unige_citations['DOI of cited article'].isna()].shape[0] / unige_citations.shape[0])

0.7927066307898374

In [15]:
# % of IARC publications with DOIs
1 - (iarc_publications.loc[iarc_publications['DOI'].isna()].shape[0] / iarc_publications.shape[0])

0.8919922949917446

In [14]:
# % of IARC citations with DOIs
1 - (iarc_citations.loc[iarc_citations['DOI of cited article'].isna()].shape[0] / iarc_citations.shape[0])

0.7314038792258963

In [16]:
# add dois
dois = unige_publications.append(iarc_publications)
dois

,Accession Number,DOI
0,WOS:000307585400002,10.1111/j.1741-2358.2011.00508.x
1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x
48,WOS:000579849900001,10.1111/ger.12502
83,WOS:000308547000129,10.1111/j.1741-2358.2011.00586.x
119,WOS:000331109200007,10.1111/ger.12083
...,...,...
275178,WOS:000365993200031,10.1016/S0140-6736(15)00128-2
275264,WOS:000418101000024,10.1016/S0140-6736(17)32129-3
275318,WOS:000467473600049,10.1038/s41586-019-1171-x
275376,WOS:000385285000007,10.1016/S0140-6736(16)31012-1


In [17]:
# rename column to add dois of citations
unige_citations = unige_citations.rename(columns = {'DOI' : 'DOI_publication', 'DOI of cited article': 'DOI'})
unige_citations

,Accession Number,DOI_publication,DOI
0,WOS:000307585400002,10.1111/j.1741-2358.2011.00508.x,NaN
1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,NaN
2,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,10.1111/j.1600-0528.1999.tb02031.x
3,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,10.1177/00220345990780020301
4,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,10.1111/j.1741-2358.2001.00079.x
...,...,...,...
1260976,WOS:000591530900014,10.1093/chemse/bjaa041,NaN
1260977,WOS:000591530900014,10.1093/chemse/bjaa041,10.1002/alr.22577
1260978,WOS:000591530900014,10.1093/chemse/bjaa041,10.1002/alr.22592
1260979,WOS:000591530900014,10.1093/chemse/bjaa041,10.1109/TPAMI.2020.2975798


In [18]:
# rename column to add dois of citations
iarc_citations = iarc_citations.rename(columns = {'DOI' : 'DOI_publication', 'DOI of cited article': 'DOI'})
iarc_citations

,Accession Number,DOI_publication,DOI
0,WOS:000492162206048,NaN,NaN
1,WOS:000438342200039,10.1093/ije/dyy122,10.1093/ije/dyy063
2,WOS:000443381100007,10.19191/EP18.1.P009.008,10.1093/ije/dyg257
3,WOS:000443381100007,10.19191/EP18.1.P009.008,NaN
4,WOS:000443381100007,10.19191/EP18.1.P009.008,NaN
...,...,...,...
275617,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN
275618,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN
275619,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN
275620,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN


In [19]:
# remove nas
unige_citations_dois = unige_citations[['Accession Number', 'DOI']].loc[unige_citations['DOI'].notna()]
unige_citations_dois

,Accession Number,DOI
2,WOS:000287362000002,10.1111/j.1600-0528.1999.tb02031.x
3,WOS:000287362000002,10.1177/00220345990780020301
4,WOS:000287362000002,10.1111/j.1741-2358.2001.00079.x
5,WOS:000287362000002,10.1161/01.STR.27.10.1812
6,WOS:000287362000002,10.1161/01.STR.24.10.1478
...,...,...
1260975,WOS:000591530900014,10.1007/s00415-010-5763-5
1260977,WOS:000591530900014,10.1002/alr.22577
1260978,WOS:000591530900014,10.1002/alr.22592
1260979,WOS:000591530900014,10.1109/TPAMI.2020.2975798


In [20]:
# remove nas
iarc_citations_dois = iarc_citations[['Accession Number', 'DOI']].loc[iarc_citations['DOI'].notna()]
iarc_citations_dois

,Accession Number,DOI
1,WOS:000438342200039,10.1093/ije/dyy063
2,WOS:000443381100007,10.1093/ije/dyg257
5,WOS:000417001900034,10.1093/carcin/bgq074
7,WOS:000417001900034,10.1006/faat.1993.1061
8,WOS:000417001900034,10.1093/aje/kwv111
...,...,...
275600,WOS:000449710900010,10.1371/journal.pmed.1001242]
275601,WOS:000449710900010,10.1080/03630242.2011.637611
275610,WOS:000449710900010,"[10.1016/S0140-6736(16)31012-1, 10.1016/S0140-..."
275616,WOS:000449710900010,10.1046/j.1360-0443.2000.95225112.x


In [21]:
# add dois
dois = dois.append(unige_citations_dois)
dois = dois.append(iarc_citations_dois)
dois

,Accession Number,DOI
0,WOS:000307585400002,10.1111/j.1741-2358.2011.00508.x
1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x
48,WOS:000579849900001,10.1111/ger.12502
83,WOS:000308547000129,10.1111/j.1741-2358.2011.00586.x
119,WOS:000331109200007,10.1111/ger.12083
...,...,...
275600,WOS:000449710900010,10.1371/journal.pmed.1001242]
275601,WOS:000449710900010,10.1080/03630242.2011.637611
275610,WOS:000449710900010,"[10.1016/S0140-6736(16)31012-1, 10.1016/S0140-..."
275616,WOS:000449710900010,10.1046/j.1360-0443.2000.95225112.x


In [22]:
# remove nas and dedup
dois = dois.loc[dois['DOI'].notna()]
dois

,Accession Number,DOI
0,WOS:000307585400002,10.1111/j.1741-2358.2011.00508.x
1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x
48,WOS:000579849900001,10.1111/ger.12502
83,WOS:000308547000129,10.1111/j.1741-2358.2011.00586.x
119,WOS:000331109200007,10.1111/ger.12083
...,...,...
275600,WOS:000449710900010,10.1371/journal.pmed.1001242]
275601,WOS:000449710900010,10.1080/03630242.2011.637611
275610,WOS:000449710900010,"[10.1016/S0140-6736(16)31012-1, 10.1016/S0140-..."
275616,WOS:000449710900010,10.1046/j.1360-0443.2000.95225112.x


In [23]:
# trim
dois['DOI'] = dois['DOI'].str.strip()
dois

C:\Users\iriarte\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Accession Number,DOI
0,WOS:000307585400002,10.1111/j.1741-2358.2011.00508.x
1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x
48,WOS:000579849900001,10.1111/ger.12502
83,WOS:000308547000129,10.1111/j.1741-2358.2011.00586.x
119,WOS:000331109200007,10.1111/ger.12083
...,...,...
275600,WOS:000449710900010,10.1371/journal.pmed.1001242]
275601,WOS:000449710900010,10.1080/03630242.2011.637611
275610,WOS:000449710900010,"[10.1016/S0140-6736(16)31012-1, 10.1016/S0140-..."
275616,WOS:000449710900010,10.1046/j.1360-0443.2000.95225112.x


In [24]:
# dedup
dois_dedup = dois.drop_duplicates(subset='DOI')
dois_dedup

,Accession Number,DOI
0,WOS:000307585400002,10.1111/j.1741-2358.2011.00508.x
1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x
48,WOS:000579849900001,10.1111/ger.12502
83,WOS:000308547000129,10.1111/j.1741-2358.2011.00586.x
119,WOS:000331109200007,10.1111/ger.12083
...,...,...
275585,WOS:000449710900010,10.1016/S0140-6736(08)61939-X
275586,WOS:000449710900010,10.1017/S0033291706009159
275588,WOS:000449710900010,10.1027/0227-5910/a000005
275592,WOS:000449710900010,10.1016/j.aap.2013.11.015


In [25]:
# reset index
dois_dedup.reset_index(drop=True, inplace=True)
dois_dedup

,Accession Number,DOI
0,WOS:000307585400002,10.1111/j.1741-2358.2011.00508.x
1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x
2,WOS:000579849900001,10.1111/ger.12502
3,WOS:000308547000129,10.1111/j.1741-2358.2011.00586.x
4,WOS:000331109200007,10.1111/ger.12083
...,...,...
751910,WOS:000449710900010,10.1016/S0140-6736(08)61939-X
751911,WOS:000449710900010,10.1017/S0033291706009159
751912,WOS:000449710900010,10.1027/0227-5910/a000005
751913,WOS:000449710900010,10.1016/j.aap.2013.11.015


In [26]:
# add ID on the first file after that it has to be obtained by DOI merge with the precedent export
# dois_dedup['ID'] = dois_dedup.index + 1
# dois_dedup

C:\Users\iriarte\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Accession Number,DOI,ID
0,WOS:000307585400002,10.1111/j.1741-2358.2011.00508.x,1
1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,2
2,WOS:000579849900001,10.1111/ger.12502,3
3,WOS:000308547000129,10.1111/j.1741-2358.2011.00586.x,4
4,WOS:000331109200007,10.1111/ger.12083,5
...,...,...,...
751910,WOS:000449710900010,10.1016/S0140-6736(08)61939-X,751911
751911,WOS:000449710900010,10.1017/S0033291706009159,751912
751912,WOS:000449710900010,10.1027/0227-5910/a000005,751913
751913,WOS:000449710900010,10.1016/j.aap.2013.11.015,751914


In [27]:
# export
dois_dedup.to_csv('export_dois_all_dedup_2.tsv', sep='\t', index=False)

## Import unpaywall data

In [28]:
import re
import os
import pandas as pd
import time
import datetime
import requests
from requests.exceptions import Timeout
dois_dedup = pd.read_csv('export_dois_without_unpaywall.tsv', sep='\t')
dois_dedup

,id,doi
0,800001,10.1111/jvh.12332_22
1,800002,10.23804/ejpd.2019.20.01.11
2,800003,10.1093/bfgp/elt032
3,800004,10.1038/ja.2017.144
4,800005,10.1016/S1368-8375(02)00024-6
...,...,...
18499,818500,10.1079/PAVSNNR20094005
18500,818501,"[10.1016/j.atherosclerosis.2019.08.014, 10.109..."
18501,818502,10.1101/326470V6
18502,818503,"[10.1016/S0140-6736(15)60401-9, 10.1016/S0140-..."


In [32]:
# import data from unpaywall
# format natifjson
for index, row in dois_dedup.iterrows():
    if (((index/100) - int(index/100)) == 0) :
        print(index)
    mydoi = row['doi']
    myid = str(row['id']).zfill(10)
    
    # test if file exists (in case of reload of errors)
    # if os.path.exists('E:/data_sources/unpaywall/eahil_2022/' + myid + '.json'):
    #     continue
    
    # start from (in case of error)
    if row['id'] < 644200 :
        continue
    # folder
    myfolder = str(int(row['id']/100000)+1)
    # 1 sec pause (not necessary, without it we don't make more than 100'000 per day)
    # time.sleep(1)
    searchurl = 'https://api.unpaywall.org/v2/' + mydoi + '?email=pablo.iriarte@unige.ch'
    headers = {'Accept': 'application/json'}
    try:
        resp = requests.get(searchurl, headers=headers, timeout=30)
    except Timeout:
        print('ERROR TIMEOUT - ID:' + myid + ' - DOI:' +  mydoi)
    else:
        # print(resp)
        if (resp.status_code == 200):
            # export
            with open('E:/data_sources/unpaywall/eahil_2022/eahil_2022_' + myfolder + '/' + myid + '.json', 'w', encoding='utf-8') as f:
                f.write(resp.text)
        else :
            print('ERROR - ID:' + myid + ' - DOI:' +  mydoi)

0
ERROR - ID:0000800002 - DOI:10.23804/ejpd.2019.20.01.11
ERROR - ID:0000800013 - DOI:10.3290/j.qi.a30771
ERROR - ID:0000800014 - DOI:10.3290/j.jad.a27795
ERROR - ID:0000800015 - DOI:10.3290/j.qi.a44635
ERROR - ID:0000800016 - DOI:10.3290/j.qi.a29751
ERROR - ID:0000800017 - DOI:10.3290/j.jad.a21852
ERROR - ID:0000800019 - DOI:10.3290/j.ohpd.a38864
ERROR - ID:0000800020 - DOI:10.3290/j.jad.a35256
ERROR - ID:0000800022 - DOI:10.3290/j.ohpd.a44682
ERROR - ID:0000800023 - DOI:10.3290/j.jad.a17548
ERROR - ID:0000800057 - DOI:10.3290/j.qi.a34701
ERROR - ID:0000800058 - DOI:10.11607/jomi.2016.1.e
ERROR - ID:0000800059 - DOI:10.3290/j.jad.a35908
ERROR - ID:0000800061 - DOI:10.3290/j.jad.a35251
ERROR - ID:0000800062 - DOI:10.3290/j.qi.a40254
ERROR - ID:0000800063 - DOI:10.3290/j.qi.a32565
ERROR - ID:0000800064 - DOI:10.1922/IDJ_2184Polzer13
ERROR - ID:0000800065 - DOI:10.3290/j.jad.a18242
ERROR - ID:0000800066 - DOI:10.3290/j.qi.a32509
ERROR - ID:0000800067 - DOI:10.3290/j.qi.a32439
ERROR - ID:

1700
ERROR - ID:0000801702 - DOI:10.1002/(SICI)1097-0258(19960229)15:4<361::AID-SIM168>3.0.CO
ERROR - ID:0000801703 - DOI:2008/23/smw-12025
ERROR - ID:0000801706 - DOI:10.1002/1097-0355(199324)14:4<330::AID-IMHJ2280140407>3.0.CO
ERROR - ID:0000801707 - DOI:10.1002/(SICI)1096-9861(19971020)387:2<167::AID-CNE1>3.0.CO
ERROR - ID:0000801708 - DOI:10.1002/(SICI)1097-4571(199401)45:1<1::AID-ASI1>3.0.CO
ERROR - ID:0000801709 - DOI:10.1002/1097-0142(195303)6:2<224::AID-CNCR2820060203>3.0.CO
ERROR - ID:0000801711 - DOI:10.1197/S1069-6563(03)00585-2
ERROR - ID:0000801712 - DOI:10.3233/978-1-61499-653-8-141
ERROR - ID:0000801713 - DOI:10.1002/1097-0142(19841215)54:12<2995::AID-CNCR2820541230>3.0.CO
ERROR - ID:0000801714 - DOI:10.1002/1097-0142(19860815)58:4<928::AID-CNCR2820580421>3.0.CO
ERROR - ID:0000801717 - DOI:10.1101/181677]
ERROR - ID:0000801720 - DOI:10.1002/(SICI)1098-2272(1999)16:1<40::AID-GEPI4>3.0.CO
ERROR - ID:0000801721 - DOI:10.1186/gb-2008-9-1-r3]
ERROR - ID:0000801722 - DOI:10.10

ERROR - ID:0000801894 - DOI:10.1002/1097-4636(2000)53:4<362::AID-JBM10>3.0.CO
ERROR - ID:0000801895 - DOI:10.1666/0094-8373(2001)027<0262:ECAAKI>2.0.CO
ERROR - ID:0000801897 - DOI:10.3290/j.ohpd.a34376
1900
ERROR - ID:0000801902 - DOI:10.3290/j.jad.a17711
ERROR - ID:0000801903 - DOI:10.2319/0003-3219(2007)077[0478:TRITPC]2.0.CO
ERROR - ID:0000801904 - DOI:10.1002/1521-4141(200106)31:6&lt
ERROR - ID:0000801905 - DOI:10.1002/(SICI)1097-0096(199910)27:8<415::AID-JCU1>3.3.CO
ERROR - ID:0000801907 - DOI:10.1002/(SICI)1521-4141(199902)29:02<499::AID-IMMU499>3.0.CO
ERROR - ID:0000801908 - DOI:10.1002/(SICI)1521-4141(199803)28:03<799::AID-IMMU799>3.0.CO
ERROR - ID:0000801909 - DOI:1184]
ERROR - ID:0000801910 - DOI:10.17795/RIJM31568]
ERROR - ID:0000801912 - DOI:2008/05/smw-11964
ERROR - ID:0000801913 - DOI:513]
ERROR - ID:0000801914 - DOI:10.14670/HH-24.879
ERROR - ID:0000801915 - DOI:10.1002/(SICI)1520-6408(1998)22:1&lt
ERROR - ID:0000801920 - DOI:10.1016/j.rcmed.2002.07.001
ERROR - ID:000080

ERROR - ID:0000802109 - DOI:10.1101/2020.03.23.20039446]
ERROR - ID:0000802111 - DOI:[10.3171/FOC- 07/11/E5, 10.3171/FOC-07/11/E5, 10.3171/FOC-07/11/E5]
ERROR - ID:0000802112 - DOI:10.1002/(SICI)1097-4571(2000)51:2<145::AID-ASI6>3.0.CO
ERROR - ID:0000802114 - DOI:09.2014/JCPSP.649652
ERROR - ID:0000802116 - DOI:10.1002/(SICI)1097-4547(19980715)53:2<260::AID-JNR15>3.3.CO
ERROR - ID:0000802118 - DOI:[10.1115/1.3658902., 10.1115/1.3658902, 10.1115/1.3658902]
ERROR - ID:0000802119 - DOI:10.1057/9780230280830_14]
ERROR - ID:0000802120 - DOI:10. 1016/b978-0-12-256420-8. 50020-4]
ERROR - ID:0000802121 - DOI:10.1007/978-3-540-27752-1]
ERROR - ID:0000802122 - DOI:[10. 1109/IEMBS. 1995. 579249, 10.1109/IEMBS.1995.579249]
ERROR - ID:0000802123 - DOI:[10. 1515/bmte. 1993. 38. s1. 79, 10.1515/BMTE.1993.38.S1.79]
ERROR - ID:0000802124 - DOI:10.1002/1097-0142(1950)3:1<32::AID-CNCR2820030106>3.0.CO
ERROR - ID:0000802126 - DOI:10.4172/2167-1079.1000132]
ERROR - ID:0000802127 - DOI:10.1002/(SICI)1096-90

ERROR - ID:0000802295 - DOI:10.7767/BOEHLAU.9783205793199.INTRO]
ERROR - ID:0000802296 - DOI:10.1007/978-3-7091-4423-7_12]
ERROR - ID:0000802298 - DOI:10.3978/j.issn.2072-1439.2016.01.19
ERROR - ID:0000802300 - DOI:10.1016/B978-0-12-397169-2.00112-8]
2300
ERROR - ID:0000802304 - DOI:10.2903/j.efsa.2015.4006]
ERROR - ID:0000802305 - DOI:[10.5799/ahinis.02.2014.03.0145, 10.5799/AHINJS.02.2014.03.0145]
ERROR - ID:0000802308 - DOI:10.1002/(SICI)1097-0169(1997)37:3&lt
ERROR - ID:0000802309 - DOI:10.16966/2380-5536.141]
ERROR - ID:0000802312 - DOI:10.1002/(SICI)1096-8628(19980401)76:4<327::AID-AJMG8>3.0.CO
ERROR - ID:0000802313 - DOI:10.1371/journal.pone.0002557]
ERROR - ID:0000802314 - DOI:10.1002/(SICI)1097-4598(199901)22:1<28::AID-MUS6>3.0.CO
ERROR - ID:0000802315 - DOI:[10.1007/978-1-4939-1438-8_, 10.1007/978-1-4939-1438-8_1]
ERROR - ID:0000802316 - DOI:10.1056/NEJMOA2001191.7
ERROR - ID:0000802318 - DOI:10.10077s00345-012-0934x
ERROR - ID:0000802321 - DOI:10.1002/(SICI)1097-0029(1998061

...

ERROR - ID:0000816093 - DOI:10.1002/(SICI)1097-0215(19970328)71:1<79::AID-IJC14>3.0.CO
ERROR - ID:0000816094 - DOI:10.1002/1097-0215(20001015)88:2&lt
ERROR - ID:0000816095 - DOI:10.1002/(SICI)1097-0215(20000601)86:5<684::AID-IJC13>3.0.CO
ERROR - ID:0000816096 - DOI:10.1002/(SICI)1097-0142(20000201)88:3<500::AID-CNCR3>3.0.CO
ERROR - ID:0000816097 - DOI:10.1700/1088.11929
ERROR - ID:0000816098 - DOI:10.5732/cjc.014.10086
ERROR - ID:0000816099 - DOI:10.1002/(SICI)1097-0274(200005)37:5<512::AID-AJIM7>3.0.CO
ERROR - ID:0000816100 - DOI:10.1002/(SICI)1097-0215(19970529)71:5<800::AID-IJC18>3.0.CO
16100
ERROR - ID:0000816101 - DOI:10.1002/1097-0142(19950515)75:10<2452::AID-CNCR2820751010>3.0.CO
ERROR - ID:0000816102 - DOI:10.1002/1097-0142(19890701)64:1&lt
ERROR - ID:0000816103 - DOI:10.1002/1097-0142(19840301)53:5<1214::AID-CNCR2820530532>3.0.CO
ERROR - ID:0000816104 - DOI:10.1002/1097-0142(197509)36:3<1077::AID-CNCR2820360335>3.0.CO
ERROR - ID:0000816105 - DOI:10.1002/1097-0142(19810301)47:5

ERROR - ID:0000816218 - DOI:10.1002/(SICI)1097-0215(19981109)78:4<446::AID-IJC10>3.3.CO
ERROR - ID:0000816221 - DOI:10.1093/JNCI/DIQ178
ERROR - ID:0000816222 - DOI:10.1002/(SICI)1097-0142(19970315)79:6<1195::AID-CNCR19>3.0.CO
ERROR - ID:0000816223 - DOI:10.1002/1097-0142(19951101)76:9<1634::AID-CNCR2820760921>3.0.CO
ERROR - ID:0000816224 - DOI:10.1002/(SICI)1097-0274(199711)32:5<535::AID-AJIM15>3.0.CO
ERROR - ID:0000816225 - DOI:10.1002/(SICI)1097-0274(199804)33:4&lt
ERROR - ID:0000816226 - DOI:10.1002/(SICI)1097-0274(199607)30:1<87::AID-AJIM15>3.0.CO
ERROR - ID:0000816227 - DOI:10.1002/(SICI)1097-0274(199602)29:2<123::AID-AJIM2>3.3.CO
ERROR - ID:0000816230 - DOI:10.4103/0973-1482.137937
ERROR - ID:0000816231 - DOI:10.1667/0033-7587(2001)156[0618:ESTTMA]2.0.CO
ERROR - ID:0000816232 - DOI:10.1667/0033-7587(2001)155[0768:PGIITY]2.0.CO
ERROR - ID:0000816233 - DOI:10.1002/1097-0215(200102)9999:9999<::AID-IJC1149>3.0.CO
ERROR - ID:0000816234 - DOI:10.1667/0033-7587(2002)157[0312:ROATAN]2.0.

ERROR - ID:0000816353 - DOI:[10.1158/1940-6207.CAPR-3-0132, 10.1158/1940-6207.CAPR-13-0132]
ERROR - ID:0000816356 - DOI:10.1002/1097-0142(19890801)64:3&lt
ERROR - ID:0000816357 - DOI:10.1039/U4WGE9EBFJN6CM8R
ERROR - ID:0000816358 - DOI:10.1002/(SICI)1097-0142(19960215)77:4<646::AID-CNCR10>3.3.CO
ERROR - ID:0000816359 - DOI:10.1002/1097-0215(20000815)87:4&lt
ERROR - ID:0000816360 - DOI:10.1002/(SICI)1097-4547(20000101)59:1<117::AID-JNR14>3.3.CO
ERROR - ID:0000816362 - DOI:10.1002/(SICI)1097-0215(19971009)73:2<208::AID-IJC8>3.0.CO
ERROR - ID:0000816363 - DOI:10.1002/1097-0142(19900901)66:5<998::AID-CNCR2820660533>3.0.CO
ERROR - ID:0000816364 - DOI:10.3109/08880018.2012.719069
ERROR - ID:0000816365 - DOI:10.22034/APJCP.2017.18.7.1751
ERROR - ID:0000816367 - DOI:10.1002/1097-0142(20010701)92:1<136::AID-CNCR1301>3.0.CO
ERROR - ID:0000816368 - DOI:10.1309/C5P6UQ2NW6LCTBP
ERROR - ID:0000816369 - DOI:10.1002/1097-0142(19870501)59:9<1665::AID-CNCR2820590924>3.0.CO
ERROR - ID:0000816370 - DOI:10

ERROR - ID:0000816488 - DOI:10.1002/(SICI)1098-2744(199612)17:4<217::AID-MC5>3.0.CO
ERROR - ID:0000816489 - DOI:10.1002/1097-0142(19901215)66:12<2569::AID-CNCR2820661220>3.0.CO
ERROR - ID:0000816492 - DOI:10.1002/(SICI)1097-0215(19970529)71:5&lt
ERROR - ID:0000816493 - DOI:10.1002/(SICI)1097-0215(19971104)73:3<356::AID-IJC9>3.0.CO
ERROR - ID:0000816494 - DOI:10.1002/(SICI)1097-0274(200002)37:2<175::AID-AJIM3>3.0.CO
ERROR - ID:0000816495 - DOI:10.1002/(SICI)1098-1004(1996)7:3<244::AID-HUMU9>3.0.CO
ERROR - ID:0000816496 - DOI:10.1002/(SICI)1098-1004(199911)14:5<440::AID-HUMU11>3.0.CO
ERROR - ID:0000816497 - DOI:10.1002/1097-0215(20000815)87:4<591::AID-IJC20>3.0.CO
ERROR - ID:0000816499 - DOI:0151809/AIM.004
ERROR - ID:0000816500 - DOI:10.1002/1097-0142(197906)43:6<2315::AID-CNCR2820430622>3.0.CO
16500
ERROR - ID:0000816501 - DOI:0141701/AIM.003
ERROR - ID:0000816504 - DOI:10.4081/ijas.2003.s1.551
ERROR - ID:0000816506 - DOI:10.1659/0276-4741(2003)023[0185:KEITEH]2.0.CO
ERROR - ID:0000816

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))